# Experiment 1.0.1

In [Experiment 1.0](experiment-1.0.ipynb), I gave ChatGPT a length of inpatient stay, plus some other simple patient characteristics, and asked it to generate a likely presenting condition and an admission note. I found that the agent's attribution of a most likely medical condition, and their reasoning in the admission note, resulted in some admissions that would be implausible in reality. For example, a patient whose visit lasted 8 days was proposed to have a fracture. The more likely outcome is that patient with a fracture would be discharged with pain control, a splint, and outpatient orthopedic follow-up for imaging and potential surgery.

Therefore, here I will deploy a second agent to critique the reasoning of the first. This way I may be able to generate more plausible presenting conditions and admission notes, that would be more consistent with an inpatient spell.

## Set up

In [1]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

In [12]:
# Load libraries
import sys
import os
import pandas as pd
from pathlib import Path

import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# Import the variables that have been set in the init.py folder in the root directory
# These include a constant called PROJECT_ROOT which stores the absolute path to this folder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import init
PROJECT_ROOT = os.getenv("PROJECT_ROOT")

# Add the src folder to sys path, so that the application knows to look there for libraries
sys.path.append(str(Path(PROJECT_ROOT) / 'src'))

# Import function to load data
from data_ingestion.load_data import load_nhse_data

from utils.write_to_json import write_to_json
from utils.load_from_json import load_from_json



## Load data

Here the function called load_file()  

* check if the NHS England datset has already been saved in a local folder called data_store in a parquet format; if so the function returns it
* if not, it checks for the zip file from the NHSE website and if not downloads it, unzips it and saves it 
* the file is unzipped to csv and read as as pandas dataframe and saved to parquet format

In [13]:
ed = load_nhse_data()
# ed.head()

In [4]:
# ed[['Age_Band', 'AE_Arrive_HourOfDay', 'AE_Time_Mins', 'Admitted_Flag', 'Length_Of_Stay_Days', 'ICD10_Chapter_Code', 'Title']].head()

## Load saved instances from experiment-1. 

I could regenerate patients, but here I want to see if I can improve on the existing admission reasons and notes. Therefore I will reload them from experiment 1. 


In [6]:
note_dict = load_from_json('note_dict')
persona_id = '3206525' # this was the fracture patient
note_dict[persona_id]

{'Age_Band': '45-64',
 'AE_Arrive_HourOfDay': '21-24',
 'AE_Time_Mins': 240,
 'Length_Of_Stay_Days': 8,
 'ICD10_Chapter_Code': 'XIX',
 'Title': 'Injury, poisoning and certain other consequences of external causes',
 'Medical_Condition': 'Fracture',
 'Admission_Note': "\n  Chief Complaint:\n  \n  The patient, a middle-aged adult, presented with severe pain and limited mobility in the right arm following a fall at home.\n\n  History of Present Illness:\n  \n  The patient reported slipping and falling on a wet floor while performing household chores. They landed on their outstretched right arm, resulting in immediate pain and inability to move the limb. There was no loss of consciousness or head injury reported. The patient experienced increasing pain and swelling in the affected arm, prompting them to seek medical attention at the Accident & Emergency Department.\n\n  Past Medical History:\n  \n  The patient's medical history is significant for hypertension, which is being managed with m

In the prompt called here, I instructed the agent to be a senior doctor whose job is to review a junior's decision to admit. The senior doctor was advised that the hospital was under severe capacity constraints, and that their job was to make sure patients were not admitted who could be managed in the community. 

In [26]:
from functions.verify_admission_decision import verify_admission_decision
verify_admission_decision(note_dict[persona_id]['Admission_Note'])

('agree',
 'In a hospital at capacity, the decision to admit versus discharge hinges on whether a condition can be safely managed outside of the hospital setting. In this case, the patient presented with severe pain and limited mobility in the right arm following a fall at home. On physical examination, there was visible swelling and deformity in the right upper extremity, specifically at the midshaft of the right humerus. Range of motion in the right arm was severely restricted due to pain. Based on these findings, it is likely that the patient has a fracture of the right humerus. Admission for further evaluation, pain management, and orthopedic consultation is appropriate to confirm the diagnosis, assess the extent of the fracture, and guide further management. Analgesics will be provided for pain control and immobilization of the right arm will be done to prevent further injury. Serial neurovascular assessments will also be conducted to monitor for any signs of compromise. Therefore

In [27]:
from functions.verify_admission_decision import verify_admission_decision
verify_admission_decision(note_dict[persona_id]['Admission_Note'])

('disagree',
 "In a hospital at capacity, the decision to admit versus discharge hinges on whether a condition can be safely managed outside of the hospital setting. In this case, the patient presents with severe pain and limited mobility in the right arm following a fall at home. The history and physical examination findings strongly support the diagnosis of a fracture of the right humerus. However, there are no signs of acute neurovascular compromise and the patient's vital signs are stable. Considering the patient's overall condition, it is reasonable to manage their fracture as an outpatient with close follow-up. Therefore, I disagree with the decision to admit the patient.")

In [28]:
from functions.verify_admission_decision import verify_admission_decision
verify_admission_decision(note_dict[persona_id]['Admission_Note'])

('disagree',
 'In a hospital at capacity, the decision to admit versus discharge hinges on whether a condition can be safely managed outside of the hospital setting. In this case, the patient presented with a fracture of the right humerus. While this is a serious injury, it can often be managed in outpatient care with adequate pain control and immobilization. The patient does not exhibit any signs of acute neurovascular compromise. Therefore, I disagree with the decision to admit the patient and recommend considering outpatient management.')

In [29]:
persona_id = '4864380' # this was the dehydration patient
note_dict[persona_id]

{'Age_Band': '85+',
 'AE_Arrive_HourOfDay': '01-04',
 'AE_Time_Mins': 630,
 'Length_Of_Stay_Days': 16,
 'ICD10_Chapter_Code': 'XVIII',
 'Title': 'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified',
 'Medical_Condition': 'Dehydration',
 'Admission_Note': "Chief Complaint:\nThe patient, an elderly individual aged 85+, presented to the Accident & Emergency Department with complaints of weakness, dizziness, and decreased urine output for the past 24 hours. \n\nHistory of Present Illness:\nThe patient reports a recent episode of vomiting and diarrhea, which has resulted in a reduced fluid intake. There is no reported fever, abdominal pain, or changes in bowel habits. The symptoms have progressively worsened over the past day.\n\nPast Medical History:\nThe patient has a past medical history of hypertension and chronic kidney disease. There is a history of recurrent urinary tract infections in the past year. No surgical history was reported. Medications i

In [30]:
from functions.verify_admission_decision import verify_admission_decision
verify_admission_decision(note_dict[persona_id]['Admission_Note'])

('agree',
 "In a hospital at capacity, the decision to admit versus discharge hinges on whether a condition can be safely managed outside of the hospital setting. In this case, the patient presented with symptoms of weakness, dizziness, and decreased urine output, which are concerning for dehydration. Considering the patient's history of chronic kidney disease and recurrent urinary tract infections, there may be an increased risk of dehydration. Admitting the patient for fluid resuscitation and close monitoring of fluid balance is appropriate to ensure optimal management and prevent complications. I agree with your decision to admit the patient.")

In [31]:
from functions.verify_admission_decision import verify_admission_decision
verify_admission_decision(note_dict[persona_id]['Admission_Note'])

('agree',
 "In a hospital at capacity, the decision to admit versus discharge hinges on whether a condition can be safely managed outside of the hospital setting. In this case, the patient presents with symptoms of weakness, dizziness, and decreased urine output, which are consistent with dehydration. The patient also has a history of chronic kidney disease and recurrent urinary tract infections, which increase the risk of dehydration. Admission for fluid resuscitation and close monitoring of fluid balance is appropriate in this situation. However, it is important to regularly reassess the patient's hydration status and response to treatment to guide the duration of the hospital stay and ensure that other options such as outpatient care are considered if appropriate.")

## Evaluation

These results suggest that I could use a second agent to evaluate the decision of the first, and then perhaps rerun whole generation step for a given persona if the admission reason does not seem plausible. That way I might end up with a more plausible set of hospital encounters, for which I would then generate further notes.

However, it is also possible that a superior version of ChatGPT (v4) would do better on the original generation task in experiment 1. Currently I'm using gpt-3.5-turbo-0613. Here I'm interested in evaluating the capability of the agent, but not necessarily attempting to accommodate every weakness. Once I have access to v4 I could retry. Therefore I will generate more notes using the approach in experiment 1, for later comparison.  